## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [59]:
import pandas as pd
import numpy as np

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
tr_types = pd.read_csv("/content/drive/MyDrive/machine_lerning/Копия tr_types.csv", sep=";")
tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/machine_lerning/Копия tr_mcc_codes.csv", on_bad_lines='skip', sep=";")
gender_train = pd.read_csv("/content/drive/MyDrive/machine_lerning/Копия gender_train.csv")
transactions = pd.read_csv("/content/drive/MyDrive/machine_lerning/Копия transactions.csv", nrows=1000000)

In [62]:
tr_types_df = pd.DataFrame(tr_types, columns=['tr_type', 'tr_description'])
tr_types_df

,tr_type,tr_description
0,3200,Плата за предоставление услуг посредством моби...
1,3210,Плата за предоставление отчета по счету карты ...
2,3800,Плата за обслуживание банковской карты (за пер...
3,4000,Плата за получение наличных в Сбербанке
4,4001,Плата за получение наличных в Сбербанке (в дру...
...,...,...
150,2990,Списание по требованию
151,2992,Списание средств для погашения задолженности п...
152,3001,Комиссия за обслуживание ссудного счета
153,3030,Плата за обслуживание банковской карты (за пос...



Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [63]:
tr_type = pd.DataFrame(transactions, columns=['tr_type'])
tr_type = tr_type.sample(n = 1000, replace=True, random_state=777)
tr_type

,tr_type
325735,2370
899887,1030
474683,1110
183206,1010
106071,1010
...,...
343356,1010
860809,1030
109094,1010
407999,7070


In [64]:
tr_type['tr_description'] = tr_type['tr_type'].map(tr_types_df.set_index('tr_type')['tr_description'])
count_atm = tr_type['tr_description'].str.contains('АТМ').sum()
count_pos = tr_type['tr_description'].str.contains('POS').sum()
print((count_atm + count_pos) * 100 / len(tr_type))
tr_type

61.5


,tr_type,tr_description
325735,2370,Списание с карты на карту по операции <перевод...
899887,1030,Оплата услуги. Банкоматы СБ РФ
474683,1110,Покупка. POS ТУ Россия
183206,1010,Покупка. POS ТУ СБ РФ
106071,1010,Покупка. POS ТУ СБ РФ
...,...,...
343356,1010,Покупка. POS ТУ СБ РФ
860809,1030,Оплата услуги. Банкоматы СБ РФ
109094,1010,Покупка. POS ТУ СБ РФ
407999,7070,Перевод на карту (с карты) через Мобильный бан...


## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [65]:
tr_type = pd.DataFrame(transactions, columns=['tr_type'])
tr_type['tr_description'] = tr_type['tr_type'].map(tr_types_df.set_index('tr_type')['tr_description'])
tr_type_count = tr_type.groupby('tr_description').count()
tr_type_count

,tr_type
tr_description,
Безналичный перевод денежных средств через POS,1237
Взнос наличных через POS,2577
Взнос наличных через POS (в своем ТБ) по счету в овердрафте,26
Взнос наличных через АТМ,1252
Взнос наличных через АТМ (в других ТБ) по счету в овердрафте,8
...,...
Списание с карты по операции “перевода с карты на карту” через POS (в пределах одного тер.банка),14
Списание с карты по операции “перевода с карты на карту” через POS (из одного ТБ в другой ТБ),4
Списание с карты по операции “перевода с карты на карту” через АТМ (в пределах одного тер.банка),9428


In [66]:
sorted_tr_type = tr_type_count.sort_values(by='tr_type', ascending=False)
top10 = sorted_tr_type.head(10)
top10

,tr_type
tr_description,
Покупка. POS ТУ СБ РФ,231117
Выдача наличных в АТМ Сбербанк России,151166
Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),149006
Покупка. POS ТУ Россия,137658
Оплата услуги. Банкоматы СБ РФ,118975
Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии),49830
Взнос наличных через АТМ (в своем тер.банке),28972
Перевод на карту (с карты) через АТМ (в пределах одного тер.банка),26078
Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя),15781


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [19]:
customers = pd.DataFrame(transactions, columns=['customer_id', 'amount'])
positive_amounts = customers[customers['amount'] > 0]
positive_amounts = positive_amounts.groupby('customer_id').sum()
customer1 = positive_amounts.sort_values(by='amount', ascending=False).head(1)
customer1

,amount
customer_id,
70780820,1.248115e+09


In [20]:
customers = pd.DataFrame(transactions, columns=['customer_id', 'amount'])
negative_amounts = customers[customers['amount'] < 0]
negative_amounts = negative_amounts.groupby('customer_id').sum()
customer2 = negative_amounts.sort_values(by='amount', ascending=True).head(1)
customer2

,amount
customer_id,
70780820,-1.249952e+09


In [167]:
receipts = customer1.at[70780820,'amount']
expenses = customer2.at[70780820,'amount']
ans = np.abs(abs(expenses) - abs(receipts))
ans

1837317.980000019

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [92]:
tr_type = pd.DataFrame(transactions, columns=['tr_type', 'amount'])
tr_type['tr_description'] = tr_type['tr_type'].map(tr_types_df.set_index('tr_type')['tr_description'])
tr_type_grouped = tr_type.groupby('tr_description').agg({'amount': ['count', 'mean', 'median']})
tr_type_grouped.columns = ['count', 'mean', 'median']

tr_type_grouped = tr_type_grouped.sort_values(by='count', ascending=False).head(10)
tr_type_grouped

,count,mean,median
tr_description,,,
Покупка. POS ТУ СБ РФ,231117,-19784.748640,-7411.52
Выдача наличных в АТМ Сбербанк России,151166,-136077.629325,-44918.32
Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя),149006,65569.831700,11319.42
Покупка. POS ТУ Россия,137658,-32119.330371,-11207.57
Оплата услуги. Банкоматы СБ РФ,118975,-5320.980222,-2245.92
Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии),49830,-205418.249032,-44918.32
Взнос наличных через АТМ (в своем тер.банке),28972,276391.789596,112295.79
Перевод на карту (с карты) через АТМ (в пределах одного тер.банка),26078,86104.332909,13951.52
Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя),15781,66806.826623,3593.47


In [184]:
customers = pd.DataFrame(transactions, columns=['customer_id', 'amount', 'tr_type'])
customers_median = customers[customers['customer_id'] == 70780820].groupby('tr_type')['amount'].median()
customers_median

tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      12992.620
7034      14306.480
7040      14396.320
7070       8714.150
7071      12543.440
7074       9387.930
Name: amount, dtype: float64

In [185]:
customers = pd.DataFrame(transactions, columns=['customer_id', 'amount', 'tr_type'])
customers_mean = customers[customers['customer_id'] == 70780820].groupby('tr_type')['amount'].mean()
customers_mean

tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
7070    1.384821e+04
7071    1.860057e+04
7074    2.281159e+04
Name: amount, dtype: float64

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [96]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

In [37]:
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
...,...,...,...,...,...,...,...,...,...
999579,76313007,188 08:03:17,6011,7015,53901.98,NaN,NaN,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в других ТБ) по счет...
999580,76313007,188 08:11:09,6011,7015,112295.79,NaN,NaN,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в других ТБ) по счет...
999581,19769561,445 10:26:16,6011,7015,463781.61,145075,NaN,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в других ТБ) по счет...
999582,701335,311 21:30:25,6011,7015,11229.58,391269,NaN,Финансовые институты — снятие наличности автом...,Взнос наличных через АТМ (в других ТБ) по счет...


1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
positive_transactions_women = transactions[(transactions["amount"] > 0) & (transactions["gender"] == 0)]
count_positive_w = positive_transactions_women.groupby("gender")["amount"].mean()

negative_transactions_women = transactions[(transactions["amount"] < 0) & (transactions["gender"] == 0)]
count_negative_w = negative_transactions_women.groupby("gender")["amount"].mean()

positive_transactions_men = transactions[(transactions["amount"] > 0) & (transactions["gender"] == 1)]
count_positive_m = positive_transactions_men.groupby("gender")["amount"].mean()

negative_transactions_men = transactions[(transactions["amount"] < 0) & (transactions["gender"] == 1)]
count_negative_m = negative_transactions_men.groupby("gender")["amount"].mean()

In [ ]:
dif1 = np.abs(count_negative_w.at[0] - count_negative_m.at[1])
dif2 = np.abs(count_positive_w.at[0] - count_positive_m.at[1])
(dif1, dif2)

(32718.054920224655, 63366.57104801464)

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [8]:
positive_transactions_women = transactions[(transactions["amount"] > 0) & (transactions["gender"] == 0)]
positive_transactions_men = transactions[(transactions["amount"] > 0) & (transactions["gender"] == 1)]

max_income_w = positive_transactions_women.groupby('tr_type')['amount'].sum().to_frame().rename(columns={'amount': 'max_income'}).sort_values(by='max_income', ascending=True).head(10)
max_income_m = positive_transactions_men.groupby('tr_type')['amount'].sum().to_frame().rename(columns={'amount': 'max_income'}).sort_values(by='max_income', ascending=True).head(10)
max_income_w

,max_income
tr_type,
4051,1122.96
4210,2245.92
4110,2245.92
2370,2335.75
7075,6737.75
2110,22459.16
1010,31532.66
4100,40334.88
2210,68315.82


In [9]:
max_income_m

,max_income
tr_type,
2020,2245.92
4110,4491.84
1010,35317.03
8100,64682.37
6000,79280.83
7041,87590.72
2370,113194.15
7015,121279.45
2110,179673.26


In [ ]:
merged_table = pd.merge(max_income_w, max_income_m, on='tr_type', how='inner')
merged_table

,max_income_x,max_income_y
tr_type,,
4110,2245.92,4491.84
2370,2335.75,113194.15
2110,22459.16,179673.26
1010,31532.66,35317.03


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [166]:
#dopolnitelnoye zadaniye

mcc_gender_amount = pd.DataFrame(transactions, columns=['gender', 'amount', 'mcc_code'])
women = mcc_gender_amount[transactions["gender"] == 0].groupby("mcc_code")['amount'].sum()
men = mcc_gender_amount[transactions["gender"] == 1].groupby("mcc_code")['amount'].sum()
#pd.merge(df1, df2, how='outer')
genders = pd.merge(women, men, on='mcc_code', how='outer')
genders['max'] = genders[['amount_x', 'amount_y']].apply(max, axis=1)
genders['gender'] = np.where(genders['amount_x'] > genders['amount_y'], '0', '1')
genders['module_dif'] = np.abs(abs(genders['amount_x']) - abs(genders['amount_y']))
genders_mcc = pd.merge(tr_mcc_codes, genders, on='mcc_code', how='outer').sort_values(by='module_dif', ascending=False).head(10)
genders_mcc = pd.DataFrame(genders_mcc, columns=['mcc_code', 'mcc_description', 'gender', 'module_dif'])
genders_mcc

,mcc_code,mcc_description,gender,module_dif
123,6010,Финансовые институты — снятие наличности вручную,1,2.674450e+09
22,4829,Денежные переводы,0,2.498985e+09
124,6011,Финансовые институты — снятие наличности автом...,0,2.196476e+09
65,5541,Станции техобслуживания,0,1.541972e+08
62,5511,"Легковой и грузовой транспорт — продажа, серви...",0,7.329935e+07
90,5812,"Места общественного питания, рестораны",0,7.032414e+07
64,5533,Автозапчасти и аксессуары,0,4.240522e+07
116,5977,Магазины косметики,1,4.240471e+07
125,6012,Финансовые институты — торговля и услуги,1,4.191607e+07
162,7995,Транзакции по азартным играм,0,4.000995e+07


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.